In [1]:
import os


from OCC.Display.WebGl.jupyter_renderer import JupyterRenderer
from OCC.Extend.DataExchange import read_step_file, export_shape_to_svg
from OCC.Display.WebGl.x3dom_renderer import X3DExporter

In [2]:
shp = read_step_file(os.path.join('..','assets', 'step', 'VentilatorAP203.step'))

In [3]:
x3dexp = X3DExporter(shp, None, None, True, (1,1,0), (0,0,0), 0, 0, (0,0,0), 2, 1)


In [4]:
x3dexp.compute()

In [5]:
len(x3dexp._triangle_sets)

1

In [6]:
x3dtriangles=x3dexp.to_x3dfile_string(0)

In [7]:
from OCC.Core.VrmlAPI import VrmlAPI_Writer

In [8]:
myWriter = VrmlAPI_Writer()

In [9]:
myWriter.Write(shp,'Vent.wrl')

True

In [10]:
from subprocess import *

In [11]:
run(['chmod', 'a+x', '../bin/tovrmlx3d'])

CompletedProcess(args=['chmod', 'a+x', '../bin/tovrmlx3d'], returncode=0)

In [12]:
x3difs=run(['../bin/tovrmlx3d','--encoding','xml','Vent.wrl'], stdout=PIPE)

In [13]:
x3domHTML = '''<script type='text/javascript' src='https://www.x3dom.org/download/dev/x3dom-full.debug.js'> </script> 
<link rel='stylesheet' type='text/css' href='https://www.x3dom.org/download/dev/x3dom.css'></link>'''

In [14]:
import xml.etree.ElementTree as ET

In [15]:
from IPython.display import HTML

In [16]:
x3domXML = ET.tostring(ET.XML(x3difs.stdout), encoding="unicode", short_empty_elements=False)

## x3dom  with VrmlAPI generated X3D

In [17]:
HTML(x3domHTML + x3domXML)

In [18]:
x3domX3Dtriangles = ET.tostring(ET.XML(x3dtriangles), encoding="unicode", short_empty_elements=False)

## x3dom with pythonocc generated X3D (triangles)

In [19]:
HTML(x3domHTML + x3domX3Dtriangles)

## Threejs baseed Jupyter renderer

In [20]:
my_renderer = JupyterRenderer()

In [21]:
my_renderer.DisplayShape(shp, render_edges=True, topo_level="Face", shape_color="#abdda4", update=True)

In [20]:
import x3d.x3d as x

x3d.py package loaded, have fun with X3D Graphics!


In [21]:
vp=x.Viewpoint(position=(-4.36117,-66.43609,65.70160),orientation=(0.99385,-0.09560,0.05589,0.74120),
               centerOfRotation=(0.00000,0.00000,0.00000),description="main view")

In [22]:
from OCC.Extend.TopologyUtils import TopologyExplorer

In [23]:
t = TopologyExplorer(shp)

In [24]:
textGeo=x.Text(fontStyle=x.FontStyle(family=['SANS'], 
                                      justify=["MIDDLE","MIDDLE"],
                                      size=5.0),
                string=["compounds: " + str(t.number_of_compounds()),
                        "shells: " + str(t.number_of_shells()),
                        "solids: " + str(t.number_of_solids()),
                        "faces: " + str(t.number_of_faces()),
                        "wires: " + str(t.number_of_wires()),
                        "vertices: " + str(t.number_of_vertices())
                       ],
              solid=False)
                

In [27]:
textShape=x.Shape(geometry=textGeo, appearance=x.Appearance(material=x.Material(diffuseColor=(0,0,1))))

In [29]:
x3dexp.write_to_file('ventilator.x3d', 0)

In [30]:
stpInline=x.Inline(url=['ventilator.x3d'])

In [32]:
scene=x.Scene(children=[vp, x.Transform(translation=(0, 0, 20), children=[textShape]), stpInline])

In [35]:
sceneXML = ET.tostring(ET.XML(scene.XML()), encoding="unicode", short_empty_elements=False)

In [37]:
HTML(x3domHTML + '<X3D>' + sceneXML + '</X3D>')